In [2]:
import os
import numpy as np

from typing import (
    List,
    Tuple,
    Optional,
    Union
)

In [23]:
def generate_bvals(num_frames: int,
                   out_file: str = 'file.bval'
                  ) -> str:
    """Creates/generates an arbitrary number b=0 b-values for a fMRI acquisition
    provided the number of dynamics/temporal frames.
    
    Usage example:
        >>> bval_file = generate_bvals(num_frames=1200, out_file="fmri.bval")
        
    Arguments:
        num_frames: Number of temporal frames/dynamics.
        out_file: Output file name.
        
    Returns:
        Output file path as string that contains the corresponding b-values.
    
    Raises:
        TypeError: Exception that is raised when the input for ``num_frames`` is not an ``int``.
    """
    if type(num_frames) != int:
        raise TypeError(f"Input for num_frames: {num_frames} is not an integer.")
        
    np.savetxt(out_file,
               np.zeros((1,num_frames),
                        dtype=np.int8,
                        order='C'),
               fmt='%i',
               delimiter=' ',
               encoding='utf-8')
    out_file: str = os.path.abspath(out_file)
    return out_file

In [25]:
def generate_bvecs(num_frames: int,
                   out_file: str = 'file.bvec'
                  ) -> str:
    """Creates/generates an arbitrary number of x,y,z b-vectors for a fMRI acquisition
    provided the number of dynamics/temporal frames.
    
    Usage example:
        >>> bval_file = generate_bvecs(num_frames=1200, out_file="fmri.bvec")
        
    Arguments:
        num_frames: Number of temporal frames/dynamics.
        out_file: Output file name.
        
    Returns:
        Output file path as string that contains the corresponding b-vectors.
    
    Raises:
        TypeError: Exception that is raised when the input for ``num_frames`` is not an ``int``.
    """
    if type(num_frames) != int:
        raise TypeError(f"Input for num_frames: {num_frames} is not an integer.")
        
    np.savetxt(out_file,
               np.ones((1,num_frames),dtype=np.int8,order='C') * np.array([[1], [0], [0]]),
               fmt='%i',
               delimiter=' ',
               encoding='utf-8')
    out_file: str = os.path.abspath(out_file)
    return out_file

In [26]:
def generate_acq_params(num_frames: int,
                        effective_echo_spacing: float = 0.05,
                        out_prefix: str = 'file'
                       ) -> Tuple[str,str]:
    """Creates acquisition parameters files for fMRI acquisition provided the number of frames and the 
    effective echo spacing.
    
    Usage example:
        >>> dist_acqp, func_acqp = generate_acq_params(num_frames=1200, 
        ...                                            effective_echo_spacing=0.05,
        ...                                            out_prefix="fmri")
        ...
        
    Arguments:
        num_frames: Number of temporal frames/dynamics.
        effective_echo_spacing: Effective echo spacing.
        out_prefix: Output file prefix.
        
    Returns:
        Tuple of strings that correspond to the acquisition parameter files (for both distortion correction and
            eddy current correction).
    
    Raises:
        TypeError: Exception that is raised when either ``num_frames`` is not an ``int`` OR when ``effective_echo_spacing`` is not a ``float``.
    """
    if (type(num_frames) != int) or (type(effective_echo_spacing) != float):
        raise TypeError(f"Input for num_frames: {num_frames} is not an integer OR effective_echo_spacing: {effective_echo_spacing} is not a float.")
    
    # Generate distortion correction
    out_dist: str = out_prefix + "_distortion_correction.acqp"
    out_func: str = out_prefix + "_functional.acqp"
    
    # Write distortion correction acqp file
    with open(out_dist,'w') as f:
        f.write(f"0 1 0 {effective_echo_spacing}\n")
        f.write(f"0 -1 0 {effective_echo_spacing}\n")
        f.close()
    
    # Write functinoal acqp file
    with open(out_func,'w') as f:
        for i in range(0,num_frames):
            f.write(f"0 1 0 {effective_echo_spacing}\n")
        f.close()
    
    # Obtain absolute file paths
    out_dist: str = os.path.abspath(out_dist)
    out_func: str = os.path.abspath(out_func)
    
    return out_dist, out_func

In [27]:
def generate_index(num_frames: int,
                   out_file: str = 'file.idx'
                  ) -> str:
    """Creates index files for use with FSL's ``eddy``.
    
    Usage example:
        >>> func_idx = generate_index(num_frames=1200, out_file="fmri.idx")
        
    Arguments:
        num_frames: Number of temporal frames/dynamics.
        out_file: Output file name.
        
    Returns:
        Output file path as string that contains the corresponding frame indices.
    
    Raises:
        TypeError: Exception that is raised when the input for ``num_frames`` is not an ``int``.
    """
    if type(num_frames) != int:
        raise TypeError(f"Input for num_frames: {num_frames} is not an integer.")
    return np.savetxt(out_file, np.ones((1, num_frames)).T, fmt="%i")

In [6]:
generate_bvals(400)

'file.bval'

In [7]:
generate_bvecs(400)

'file.bvec'

In [23]:
generate_index(400)

In [28]:
generate_acq_params(0.05,400)

('file_distortion_correction.acqp', 'file_functional.acqp')

In [215]:
# slices = 45
slices = 15

In [216]:
# mb_factor = 9
mb_factor = 3

In [217]:
locs = slices//mb_factor
locs

5

In [218]:
step = int(np.round(np.sqrt(locs)))
step
# step = 1

2

In [219]:
# This works for interleaved data
n = []
for s in range(step):
    for k in range(s, locs, step):
        if mb_factor != 1:
            a = [k + locs*j for j in range(mb_factor)]
            n.append(a)
        else:
            a = k
            n.append(a)
n = np.array(n)
# n = np.transpose(n)
n

array([[ 0,  5, 10],
       [ 2,  7, 12],
       [ 4,  9, 14],
       [ 1,  6, 11],
       [ 3,  8, 13]])

In [132]:
slices = 45
mb_factor = 9

In [133]:
locs = slices//mb_factor
locs

5

In [139]:
# step = int(np.round(np.sqrt(locs)))
# step
step = 1

In [140]:
# This single shot data
n = []
for s in range(step):
    # print(s)
    for k in range(locs):
        # print(k)
        if mb_factor != 1:
            a = [k + locs*n for n in range(mb_factor)]
            n.append(a)
        else:
            a = k
            n.append(a)
    n = np.array(n)
# n = np.transpose(n)
n

array([[ 0,  5, 10, 15, 20, 25, 30, 35, 40],
       [ 1,  6, 11, 16, 21, 26, 31, 36, 41],
       [ 2,  7, 12, 17, 22, 27, 32, 37, 42],
       [ 3,  8, 13, 18, 23, 28, 33, 38, 43],
       [ 4,  9, 14, 19, 24, 29, 34, 39, 44]])

In [20]:
def generate_slice_order(slices: int,
                         mb_factor: int = 1,
                         mode: str = 'interleaved',
                         out_file: str = 'file.slice.order',
                         return_mat: Optional[bool] = False
                        ) -> Union[str,np.array]:
    """Generates the slice acquisition order file for use with ``eddy's`` slice-to-volume motion correction method.
    
    The file generated consists of an (N/m) x m matrix | N = number of slices in the acquisition direction (assumed to be
    the z-direction), and m = multi-band factor.
    
    NOTE:
        Links for details:
            * Documentation: https://fsl.fmrib.ox.ac.uk/fsl/fslwiki/eddy/UsersGuide#A--slspec
            * Implementation: https://git.fmrib.ox.ac.uk/seanf/dhcp-neonatal-fmri-pipeline/-/blob/master/dhcp/resources/default_func.slorder
            * Implementation: https://git.fmrib.ox.ac.uk/matteob/dHCP_neo_dMRI_pipeline_release/-/blob/master/slorder.txt
    
    Usage example:
        >>> sls_order = generate_slice_order(slices=44,
        ...                                  mb_factor=6,
        ...                                  mode='single-shot',
        ...                                  out_file='file.slice.order',
        ...                                  return_mat=False)
        ...
        
    Arguments:
        slices: Number of slices in the acquisition direction.
        mb_factor: Multi-band factor.
        mode: Acquisition algorithm/method/scheme used to acquire the data. Valid options include:
            * ``interleaved``: Optimal slice acquisition technique in which non-adjacent slices are acquired (best for diffusion MR images).
            * ``single-shot``: Slice acquisition in which slices are acquired sequentially with an ascending slice order (best for functional MR images).
            * ``default``: Default acquisition order in which slices are acquired with an ascending slice order.
        out_file: Output file name.
        return_mat: Return a numpy multi-dimensional array (matrix) rather than a file.
        
    Returns:
        Output file path as string that contains the slice acquisition order OR a numpy multi-dimensional array if specified.
    
    Raises:
        TypeError: Exception that is raised in the case that either ``slices`` or ``mb_factor`` is not an ``int``.
        ValueError: Exception that is raised in the case that neither ``interleaved``, ``single-shot``, or ``default`` is passed as an argument for ``mode``.
    """
    # Check input types
    if (type(slices) != int) or (type(mb_factor) != int):
        raise TypeError(f"Input option slices: {slices} or mb_factor: {mb_factor} is not an integer.")
    
    # Locations (in the slices) divided by Multi-Band Factor
    locs: int = slices//mb_factor
    
    if mode == 'interleaved':
        step: int = int(np.round(np.sqrt(locs)))
    elif mode == 'default':
        step: int = 2
    elif mode == 'single-shot':
        step: int = 1
    else:
        raise ValueError(f"Option specified for mode: {mode} does not exist.")
    
    # Iterate through each MB acquisition to get slice ordering
    n: List[int] = []
    
    for s in range(step):
        for k in range(s, locs, step):
            if mb_factor != 1:
                a: List[int] = [ k + locs*j for j in range(mb_factor) ]
                n.append(a)
            else:
                a: int = k
                n.append(a)
    
    if return_mat:
        return np.array(n)
    
    slice_order: np.arrary = np.array(n)
    np.savetxt(out_file,
               slice_order, 
               fmt="%i")
    return out_file

In [4]:
k = 5

In [5]:
type(k).as_integer_ratio()

int

(100, 1)

In [35]:
n = np.array([[0,1,2,3,4]]).T
# n = np.transpose(n).T
n

array([[0],
       [1],
       [2],
       [3],
       [4]])

In [36]:
n.shape

(5, 1)

In [35]:
mb_factor = 4

In [36]:
slices = 44

In [47]:
a = []
for i in range(0,mb_factor):
    # print(i)
    n = []
    for j in range(i,slices,mb_factor):
        n.append(j)
    n = np.array([n])
    a.append(n)
# a = tuple(a)
# a = np.concatenate(a)
a

[array([[ 0,  4,  8, 12, 16, 20, 24, 28, 32, 36, 40]]),
 array([[ 1,  5,  9, 13, 17, 21, 25, 29, 33, 37, 41]]),
 array([[ 2,  6, 10, 14, 18, 22, 26, 30, 34, 38, 42]]),
 array([[ 3,  7, 11, 15, 19, 23, 27, 31, 35, 39, 43]])]

In [ ]:
# for i in range(0,)

In [45]:
# Looks cool - but was the incorrect method - this always assumed that the MB factor was 9.
# 
# 
# def generate_slice_order(slices: int,
#                          mb_factor: int = 1,
#                          out_file: str = 'file.slice.order',
#                          return_mat: Optional[bool] = False
#                         ) -> Union[str,np.array]:
#     """working doc-string
    
#     link for details: https://git.fmrib.ox.ac.uk/seanf/dhcp-neonatal-fmri-pipeline/-/blob/master/dhcp/resources/default_func.slorder
#     """
#     try:
#         slice_order: List[int] = []
#         for i in range(0,mb_factor):
#             n: List[int] = []
#             for j in range(i,slices+1,mb_factor):
#                 n.append(j)
#             n: np.array = np.array([n])
#             slice_order.append(n)
        
#         if return_mat:
#             return np.concatenate(slice_order)
        
#         slice_order: np.arrary = np.concatenate(slice_order)
#         np.savetxt(out_file,
#                    slice_order, 
#                    fmt="%i")
#         return out_file
#     except ValueError:
#         return generate_slice_order(slices=slices-1,
#                                     mb_factor=mb_factor,
#                                     return_mat=return_mat)

In [35]:
slice_order_mat: np.array = generate_slice_order(slices=45,mb_factor=9,mode='single-shot',return_mat=True)
slice_order_mat

array([[ 0,  5, 10, 15, 20, 25, 30, 35, 40],
       [ 1,  6, 11, 16, 21, 26, 31, 36, 41],
       [ 2,  7, 12, 17, 22, 27, 32, 37, 42],
       [ 3,  8, 13, 18, 23, 28, 33, 38, 43],
       [ 4,  9, 14, 19, 24, 29, 34, 39, 44]])

In [36]:
# Define the mp_order (Movement parameter order)
mp_order: int = slice_order_mat.shape[0]-1
mp_order

4

In [2]:
import numpy as np
import nibabel as nib

In [3]:
nii_file = "b0s.nii.gz"

In [4]:
img = nib.load(nii_file)

In [10]:
img.header['dim'][3]

45

In [11]:
2%2

0

In [12]:
def _is_even(num: int) -> bool:
    """working doc-string
    """
    if (num%2 == 0):
        return True
    else:
        return False

In [14]:
_is_even(44)

True

# Test ``utils`` implementation details

In [1]:
import os
import nibabel as nib

In [2]:
import util

In [4]:
t = util.File(file="")

In [5]:
t

In [3]:
nii_file = 'test.func.nii.gz'

In [10]:
# with util.NiiFile(nii_file) as f:
#     # print(f.abs_path())
#     print(f.file_parts())

In [5]:
f = util.NiiFile(nii_file)

In [9]:
# f.write_txt(txt="some text",header_field=util.NiiHeaderField.intent_name)

In [8]:
# f.touch()

In [11]:
import sys

In [ ]:
# New implementation
sys.getsizeof(f)

In [12]:
# Old implementation
sys.getsizeof(f)

48

In [9]:
f = util.NiiFile(nii_file)

In [10]:
f_header = nib.load(f.file)

In [21]:
print(f_header)

<class 'nibabel.nifti1.Nifti1Image'>
data shape (64, 64, 44, 400)
affine: 
[[ -2.5          0.           0.          73.41090393]
 [  0.           2.5          0.         -35.51405334]
 [  0.           0.           2.5        -34.53044128]
 [  0.           0.           0.           1.        ]]
metadata:
<class 'nibabel.nifti1.Nifti1Header'> object, endian='<'
sizeof_hdr      : 348
data_type       : b''
db_name         : b''
extents         : 16384
session_error   : 0
regular         : b'r'
dim_info        : 0
dim             : [  4  64  64  44 400   1   1   1]
intent_p1       : 0.0
intent_p2       : 0.0
intent_p3       : 0.0
intent_code     : none
datatype        : float32
bitpix          : 32
slice_start     : 0
pixdim          : [-1.         2.5        2.5        2.5        1.1997647  0.
  0.         0.       ]
vox_offset      : 0.0
scl_slope       : nan
scl_inter       : nan
slice_end       : 0
slice_code      : unknown
xyzt_units      : 10
cal_max         : 0.0
cal_min         : 0

In [23]:
f_header.header['intent_name'] = "Test string is this greater than 16-bytes?"

In [24]:
nib.save(f_header,f.file)

In [25]:
len("Test string is t")

16

In [5]:
f.ext

'.nii.gz'

In [4]:
work = util.WorkDir(work_dir="test.workdir",use_cwd=True)

In [7]:
work.mkdir()

In [8]:
work.rmdir()

In [5]:
tmp = util.TmpDir(tmp_dir=os.getcwd(),use_cwd=True)

In [6]:
tmp

/Users/adebayobraimah/Desktop/projects/fmri_preproc/tmp_dir_1153

In [7]:
tmp.parent_dir

'/Users/adebayobraimah/Desktop/projects/fmri_preproc'

In [11]:
tmp.mkdir()

In [14]:
tmp_file = util.TmpDir.TmpFile(tmp_dir=tmp.tmp_dir,ext='nii.gz')

In [15]:
tmp_file

/Users/adebayobraimah/Desktop/projects/fmri_preproc/tmp_dir_8434/tmp_file_1682.nii.gz

In [16]:
tmp_file.touch()

In [15]:
tmp.rmdir()

# Define `Enums`

In [2]:
from enum import(
    Enum,
    unique
)

In [3]:
@unique
class NiiHeaderField(Enum):
    descrip: int = 1
    intent_name: int = 2

In [5]:
NiiHeaderField.descrip == NiiHeaderField.descrip

True

In [7]:
NiiHeaderField.ssss

AttributeError: ssss

In [8]:
import sys

In [15]:
ff = util.File(file="test.file.txt")

In [ ]:
# New implementation
sys.getsizeof(ff)

In [16]:
# Old implementation
sys.getsizeof(ff)

48

In [1]:
from fslpy import eddy
from util import Command

In [2]:
eddy("","","","","","","")

command: eddy_cuda7.5 
 Failed with returncode -6
ERROR: dyld: Library not loaded: @rpath/libcublas.7.5.dylib
  Referenced from: /usr/local/fsl/6.0.5/bin/eddy_cuda7.5
  Reason: image not found

command: eddy 
 Failed with returncode 1
ERROR: ***************************************************
The following COMPULSORY options have not been set:
	--imain	File containing all the images to estimate distortions for
	--mask	Mask to indicate brain
	--index	File containing indices for all volumes in --imain into --acqp and --topup
	--acqp	File containing acquisition parameters
	--bvecs	File containing the b-vectors for all volumes in --imain
	--bvals	File containing the b-values for all volumes in --imain
	--out	Basename for output
***************************************************

Part of FSL (ID: 6.0.5:9e026117)
eddy 
Copyright(c) 2015, University of Oxford (Jesper Andersson)

Usage: 
eddy --monsoon

Compulsory arguments (You MUST set one or more of):
	--imain	File containing all the image

In [7]:
p = Command("eddy")
# p.cmd_list.append("-h")
p.run()

command: eddy 
 Failed with returncode 1
ERROR: ***************************************************
The following COMPULSORY options have not been set:
	--imain	File containing all the images to estimate distortions for
	--mask	Mask to indicate brain
	--index	File containing indices for all volumes in --imain into --acqp and --topup
	--acqp	File containing acquisition parameters
	--bvecs	File containing the b-vectors for all volumes in --imain
	--bvals	File containing the b-values for all volumes in --imain
	--out	Basename for output
***************************************************

Part of FSL (ID: 6.0.5:9e026117)
eddy 
Copyright(c) 2015, University of Oxford (Jesper Andersson)

Usage: 
eddy --monsoon

Compulsory arguments (You MUST set one or more of):
	--imain	File containing all the images to estimate distortions for
	--mask	Mask to indicate brain
	--index	File containing indices for all volumes in --imain into --acqp and --topup
	--acqp	File containing acquisition parameters
	-

(1, None, None)

In [1]:
help("keywords")


Here is a list of the Python keywords.  Enter any keyword to get more help.

False               class               from                or
None                continue            global              pass
True                def                 if                  raise
and                 del                 import              return
as                  elif                in                  try
assert              else                is                  while
async               except              lambda              with
await               finally             nonlocal            yield
break               for                 not                 



In [3]:
help()


Welcome to Python 3.8's help utility!

If this is your first time using Python, you should definitely check out
the tutorial on the Internet at https://docs.python.org/3.8/tutorial/.

Enter the name of any module, keyword, or topic to get help on writing
Python programs and using Python modules.  To quit this help utility and
return to the interpreter, just type "quit".

To get a list of available modules, keywords, symbols, or topics, type
"modules", "keywords", "symbols", or "topics".  Each module also comes
with a one-line summary of what it does; to list the modules whose name
or summary contain a given string such as "spam", type "modules spam".


You are now leaving help and returning to the Python interpreter.
If you want to ask for help on a particular object directly from the
interpreter, you can type "help(object)".  Executing "help('string')"
has the same effect as typing a particular string at the help> prompt.


In [4]:
help("symbols")


Here is a list of the punctuation symbols which Python assigns special meaning
to. Enter any symbol to get more help.

!=                  +                   <=                  __
"                   +=                  <>                  `
"""                 ,                   ==                  b"
%                   -                   >                   b'
%=                  -=                  >=                  f"
&                   .                   >>                  f'
&=                  ...                 >>=                 j
'                   /                   @                   r"
'''                 //                  J                   r'
(                   //=                 [                   u"
)                   /=                  \                   u'
*                   :                   ]                   |
**                  <                   ^                   |=
**=                 <<                  ^=                  ~
*=

In [5]:
help("topics")


Here is a list of available topics.  Enter any topic name to get more help.

ASSERTION           DELETION            LOOPING             SHIFTING
ASSIGNMENT          DICTIONARIES        MAPPINGMETHODS      SLICINGS
ATTRIBUTEMETHODS    DICTIONARYLITERALS  MAPPINGS            SPECIALATTRIBUTES
ATTRIBUTES          DYNAMICFEATURES     METHODS             SPECIALIDENTIFIERS
AUGMENTEDASSIGNMENT ELLIPSIS            MODULES             SPECIALMETHODS
BASICMETHODS        EXCEPTIONS          NAMESPACES          STRINGMETHODS
BINARY              EXECUTION           NONE                STRINGS
BITWISE             EXPRESSIONS         NUMBERMETHODS       SUBSCRIPTS
BOOLEAN             FLOAT               NUMBERS             TRACEBACKS
CALLABLEMETHODS     FORMATTING          OBJECTS             TRUTHVALUE
CALLS               FRAMEOBJECTS        OPERATORS           TUPLELITERALS
CLASSES             FRAMES              PACKAGES            TUPLES
CODEOBJECTS         FUNCTIONS           POWER           

In [6]:
l_var = ["list","of","stings"]

In [7]:
l_var.append(9)

In [8]:
l_var

['list', 'of', 'stings', 9]

In [40]:
tr = 2.5
hz = 0

In [31]:
from math import (
    e as EULER_CONST,
    log,
    sqrt
)

In [43]:
fwhm_kernel: float = sqrt(8*log(2,EULER_CONST))
try:
    sigma: float = 1/(fwhm_kernel * tr * hz)
except ZeroDivisionError:
    sigma: float = 0.0
sigma

0.0

In [38]:
1/0.009

111.11111111111111

In [16]:
# help("//")

In [39]:
1/111

0.009009009009009009

In [1]:
from fslpy import fslmaths

In [2]:
c = fslmaths(img="test.nii.gz").bptf(high_pass=100,low_pass=0,input_is_sec=True,tr=2.00)

In [12]:
# c.run(out='tmp.nii.gz')

In [1]:
from enums import FSLDataType

In [2]:
FSLDataType.char.name

'char'

In [4]:
FSLDataType("double").name

'double'

In [5]:
FSLDataType("input").name

'input'

In [1]:
# from enums import ECOLType

In [3]:
# ECOLType.sw

<ECOLType.sw: 'sw'>

In [4]:
from enum import Enum

In [11]:
class T(Enum):
    t = 1
    v = 2

In [12]:
T(1).name

't'

In [1]:
from util import NiiFile

In [2]:
f = NiiFile("test.file")

In [5]:
f.file

'test.file.nii.gz'

In [6]:
from enum import Enum

In [7]:
class class1:
    class class2(Enum):
        v = 1
        t = 2
    class class3(Enum):
        g = 1
        b = 2

In [10]:
class1.class2.v

<class2.v: 1>

In [1]:
from util import (
    File,
    WorkDir,
    TmpDir,
    NiiFile,
    Command
)
import sys

In [2]:
f = File("file.bval",assert_exists=True)

In [3]:
sys.getsizeof(f)

48

In [4]:
ff = TmpDir(tmp_dir="NewDir",use_cwd=True)

In [5]:
# With __slots__
sys.getsizeof(ff)

64

In [5]:
# W/o __slots__
sys.getsizeof(ff)

72

In [6]:
ff.work_dir

'/Users/adebayobraimah/Desktop/projects/fmri_preproc/NewDir/tmp_dir_4049'

In [7]:
f1 = WorkDir('testDir',use_cwd=True)

In [8]:
sys.getsizeof(f1)

48

In [9]:
tmp_ff_file = ff.TmpFile(tmp_dir=ff.work_dir)

In [10]:
tmp_ff_file.__getattribute__

<method-wrapper '__getattribute__' of TmpFile object at 0x7ff651ba07c0>

In [11]:
sys.getsizeof(tmp_ff_file)

64

In [13]:
tt = NiiFile(file="test.nii.gz",assert_exists=True,validate_nifti=True)

In [17]:
t2 = NiiFile(file="test2.nii.gz",assert_exists=True,validate_nifti=False)

In [14]:
tt.write_txt("some text","descrip")

In [19]:
# t2.write_txt("some text?")

In [14]:
tt.ext

'.nii.gz'

In [15]:
sys.getsizeof(tt)

64

In [16]:
# Create empty image file
! fslcreatehd 100 100 100 1 1 1 1 1 0 0 0 8 test.nii.gz

In [7]:
c = Command("fslhd")

In [3]:
sys.getsizeof(c)

48

In [4]:
c.check_dependency()

True

In [5]:
c.cmd_list.append("-h")

In [8]:
c.run()

command: fslhd 
 Failed with returncode 1


(1, None, None)

In [3]:
from util import Command
import sys

In [2]:
c = Command("fslhd")

In [4]:
sys.getsizeof(c)

48

In [5]:
c.check_dependency()

True

In [6]:
c.append("-h")

In [7]:
c.run()

command: fslhd -h 
 Failed with returncode 1
ERROR: Image Exception : #63 :: No image files match: -h
No image files match: -h



(1, None, None)

In [13]:
c = Command("echo")

In [14]:
c.append("Hi!")
c.append("I have arrived!")

In [15]:
c.run(shell=True)

(0, None, None)

In [16]:
c

echo Hi! I have arrived!

In [2]:
ics = [ 0, 2, 3, 1 ,4 ,5 ]

In [7]:
icstr: str = '"'
for i in range(0, len(ics)-1):
    icstr: str = icstr + f"{ics[i]},"
icstr: str = icstr + f'{ics[-1]}"'
icstr

'"0,2,3,1,4,5,5"'

In [1]:
from mcdc import generate_slice_order

In [6]:
generate_slice_order(slices=44,
                     mb_factor=4,
                     mode='single-shot',
                     out_file=None,
                     return_mat=True)

array([[ 0, 11, 22, 33],
       [ 1, 12, 23, 34],
       [ 2, 13, 24, 35],
       [ 3, 14, 25, 36],
       [ 4, 15, 26, 37],
       [ 5, 16, 27, 38],
       [ 6, 17, 28, 39],
       [ 7, 18, 29, 40],
       [ 8, 19, 30, 41],
       [ 9, 20, 31, 42],
       [10, 21, 32, 43]])

In [1]:
from util import File
import os

In [2]:
f = File(file=os.getcwd()+'/test',ext=None,assert_exists=False)

In [3]:
f.ext

''

In [4]:
f

/Users/adebayobraimah/Desktop/projects/fmri_preproc/test

In [5]:
t = "test.cnf"

In [6]:
t.endswith(".*")

False

In [7]:
os.path.splitext(f.file)

('/Users/adebayobraimah/Desktop/projects/fmri_preproc/test', '')

In [8]:
os.path.splitext(t)

('test', '.cnf')

In [9]:
f.file_parts()

('/Users/adebayobraimah/Desktop/projects/fmri_preproc', 'test', '')

In [1]:
from util import NiiFile

In [2]:
nii = "/Users/adebayobraimah/Desktop/projects/fmri_preproc/data/sub-194/functional/sub-194/func/sub-194_task-rest_dir-PA_run-01_bold.nii.gz"

In [4]:
n = NiiFile(file=nii,assert_exists=True,validate_nifti=True)

In [6]:
from fslpy import fslmaths

In [8]:
fslmaths(img=n.file).add(input=1).mul(input=2).div(input=2.2).run(out="test.2.nii.gz")

'test.2.nii.gz'

In [9]:
f = NiiFile(file='test.2.nii.gz',validate_nifti=True)

In [10]:
verbose = 0

In [11]:
if verbose:
    print(True)